In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import os
from mpl_toolkits.mplot3d import axes3d
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
import sklearn.naive_bayes as nb
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from tensorflow.keras.layers import LSTM,Conv1D,MaxPooling1D
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
import tensorflow.keras.backend as K 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,Input,Dense,Activation,Flatten,SeparableConv2D,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tqdm import tqdm
from sklearn.utils import resample

/home/younghwan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/younghwan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/younghwan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/younghwan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dty

In [2]:
target_folder = 'swing'
mode = 'LSTM model'

print('this model is ',mode,'\n\n')
filename_in_dir = []

for root, dirs, files in os.walk(target_folder):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)
print('load_file_complete')
print('make DataFrame...')
total_data = []
total_label = []

for z in tqdm(filename_in_dir):
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp.append(df['GX'][i]/1000)
        tmp.append(df['GY'][i]/1000)
        tmp.append(df['GZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
print('make DataFrame complete ....')

print('label_encoding processing...')
encoder = LabelEncoder()
arr_label = encoder.fit_transform(total_label)

total_data = np.array(total_data)
arr_label = np.array(arr_label)
print('Done....')
print(len(arr_label))

  0%|          | 8/12331 [00:00<02:53, 71.03it/s]

this model is  LSTM model 


load_file_complete
make DataFrame...


100%|██████████| 12331/12331 [02:25<00:00, 84.75it/s] 


make DataFrame complete ....
label_encoding processing...
Done....
12331


In [3]:
tmp_li = []
tmp_li.append(0)
for x in range(len(arr_label)-1):
    if arr_label[x]!=arr_label[x+1]:
        tmp_li.append(x)
tmp_li.append(len(arr_label)-1)
print(tmp_li)  

[0, 1838, 3407, 5041, 6741, 8170, 9491, 10885, 12330]


In [13]:
div_data = []
div_label = []
for x in range(len(tmp_li)-1):
    div_tmp =[]
    div_la = []
    for y in range(tmp_li[x]+1,tmp_li[x+1]+1):
        div_tmp.append(total_data[y])
        div_la.append(arr_label[y])
    div_data.append(div_tmp)
    div_label.append(div_la)


In [14]:

boot = []
label = []
t_li = [5,1,7,6,0,4,2,3]
for x in range(len(div_data)):
    tmp = resample(div_data[x],replace=True,n_samples = 5000,random_state=1)
    boot+=tmp
    label += list(t_li[x] for i in range(5000))
    

In [15]:
boot = np.array(boot)
label = np.array(label)

total_data = np.append(total_data,boot,axis=0)
arr_label = np.append(arr_label,label,axis=0)

x_train,x_test,y_train,y_test = train_test_split(total_data,arr_label,test_size=0.2,random_state=0)

nsamples, nx, ny = x_train.shape
d2_train_dataset = x_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

config = tf.ConfigProto() 
config.gpu_options.per_process_gpu_memory_fraction = 0.9 
session = tf.Session(config=config)



with tf.device('/GPU:0'):
    model = Sequential() # Sequeatial Model 
    model.add(LSTM(180, input_shape=(60,6),return_sequences = True)) # (timestep, feature) 
    model.add(Dropout(0.8))
    model.add(Conv1D(512,
                     2,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(MaxPooling1D(pool_size=4))
    model.add(LSTM(128))
    model.add(Dense(8, activation='softmax'))

    # 3. 모델 학습과정 설정하기
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist = model.fit(x_train, y_train, epochs=100, batch_size=64, validation_data=(x_test, y_test))
    model.save('model.h5')
    model.save_weights('model_weights.h5')
print("Best: " ,(hist.history['acc']))

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp.append(df['GX'][i]/1000)
        tmp.append(df['GY'][i]/1000)
        tmp.append(df['GZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
total_data = np.array(total_data)
nsamples, nx, ny = total_data.shape
#sample = total_data.reshape((nsamples,nx*ny))

sample_pred = model.predict(total_data)
sample_pred = np.argmax(sample_pred,axis=-1)
lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(total_label[x],' --> ' ,lab[x],'        err_index number : ',x)

        
print('hit: ',hit,' miss : ',miss,'percent : ',(100*hit)/(hit+miss))








1 Physical GPUs, 1 Logical GPUs
Train on 41864 samples, validate on 10467 samples
Epoch 1/100
41864/41864 [==============================] - 38s 907us/sample - loss: 0.1632 - acc: 0.9458 - val_loss: 0.0770 - val_acc: 0.9795
Epoch 2/100
41864/41864 [==============================] - 37s 891us/sample - loss: 0.0630 - acc: 0.9812 - val_loss: 0.0449 - val_acc: 0.9882
Epoch 3/100
41864/41864 [==============================] - 37s 889us/sample - loss: 0.0418 - acc: 0.9876 - val_loss: 0.0423 - val_acc: 0.9866
Epoch 4/100
41864/41864 [==============================] - 37s 891us/sample - loss: 0.0338 - acc: 0.9899 - val_loss: 0.0280 - val_acc: 0.9924
Epoch 5/100
41864/41864 [==============================] - 37s 890us/sample - loss: 0.0257 - acc: 0.9920 - val_loss: 0.0292 - val_acc: 0.9919
Epoch 6/100
41864/41864 [==============================] - 37s 889us/sample - loss: 0.0210 - acc: 0.9933 - val_loss: 0.0305 - val_acc: 0.9893
Epoch 7/100
41864/41864 [==============================] - 37s 886

In [16]:
#####here to change####
######################################################################


best_score = 0

for gamma in [0.001,0.01,0.1,1,10,100]:
    for C in [0.001,0.01,0.1,1,10,100]:
        for kernel in ['linear','rbf','poly']:
            tmp_model = svm.SVC(kernel=kernel,gamma=gamma,C=C)
            scores = cross_val_score(tmp_model,d2_train_dataset,y_train,cv=10,n_jobs=-1)
            score = np.mean(scores)
            
            if score>best_score:
                
                best_score = score
                best_parameter = {'kernel':kernel,'gamma':gamma,'C':C}
                print('best_parameter is change : ',best_parameter)
            else:
                print('remain :',best_parameter)
mod = svm.SVC(**best_parameter)


######################################################################


predict_model = mod.fit(d2_train_dataset,y_train)
print('fitting ',mode,' is complete...')
print(mode,'score is :',predict_model.score(d2_test_dataset,y_test))

prediction = predict_model.predict(d2_test_dataset)



hit = 0
miss = 0
for x in range(len(y_test)):
    if prediction[x] == y_test[x]:
        hit+=1
    else:
        miss+=1

print('Validation Result...')
print('hit: ',hit,' miss : ',miss)

print('\n\nnow test new data..')

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp.append(df['GX'][i]/1000)
        tmp.append(df['GY'][i]/1000)
        tmp.append(df['GZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
total_data = np.array(total_data)
nsamples, nx, ny = total_data.shape
sample = total_data.reshape((nsamples,nx*ny))

sample_pred = predict_model.predict(sample)

lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(total_label[x],' --> ' ,lab[x],'err_index : ',x)

print('hit: ',hit,' miss : ',miss)

best_parameter is change :  {'kernel': 'linear', 'gamma': 0.001, 'C': 0.001}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.001}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.001}
best_parameter is change :  {'kernel': 'linear', 'gamma': 0.001, 'C': 0.01}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.01}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.01}
best_parameter is change :  {'kernel': 'linear', 'gamma': 0.001, 'C': 0.1}


KeyboardInterrupt: 

In [ ]:
#####here to change####
######################################################################

space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

mod =xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = space['max_depth'], gamma = space['gamma'],
                    reg_alpha = space['reg_alpha'],min_child_weight=space['min_child_weight'],
                    colsample_bytree=space['colsample_bytree'])
    
######################################################################


predict_model = mod.fit(d2_train_dataset,y_train)
print('fitting ',mode,' is complete...')
print(mode,'score is :',predict_model.score(d2_test_dataset,y_test))

prediction = predict_model.predict(d2_test_dataset)



hit = 0
miss = 0
for x in range(len(y_test)):
    if prediction[x] == y_test[x]:
        hit+=1
    else:
        miss+=1

print('Validation Result...')
print('hit: ',hit,' miss : ',miss)

print('\n\nnow test new data..')

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp.append(df['GX'][i]/1000)
        tmp.append(df['GY'][i]/1000)
        tmp.append(df['GZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
total_data = np.array(total_data)
nsamples, nx, ny = total_data.shape
sample = total_data.reshape((nsamples,nx*ny))

sample_pred = predict_model.predict(sample)

lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(total_label[x],' --> ' ,lab[x],'err_index : ',x)

print('hit: ',hit,' miss : ',miss)

In [ ]:
##위에거는 xg boost

In [ ]:
#####here to change####
######################################################################


mod = GaussianNB()


######################################################################


predict_model = mod.fit(d2_train_dataset,y_train)
print('fitting ',mode,' is complete...')
print(mode,'score is :',predict_model.score(d2_test_dataset,y_test))

prediction = predict_model.predict(d2_test_dataset)



hit = 0
miss = 0
for x in range(len(y_test)):
    if prediction[x] == y_test[x]:
        hit+=1
    else:
        miss+=1

print('Validation Result...')
print('hit: ',hit,' miss : ',miss)

print('\n\nnow test new data..')

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp.append(df['GX'][i]/1000)
        tmp.append(df['GY'][i]/1000)
        tmp.append(df['GZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
total_data = np.array(total_data)
nsamples, nx, ny = total_data.shape
sample = total_data.reshape((nsamples,nx*ny))

sample_pred = predict_model.predict(sample)

lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(total_label[x],' --> ' ,lab[x],'err_index : ',x)

print('hit: ',hit,' miss : ',miss)

In [ ]:
#위에거는 나이브 베이즈

In [ ]:
#####here to change####
######################################################################
rfc=RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [10,15,20,30,40,50,100,200,300,500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [3,4,5,6,7,8,9,10,11,12],
    'criterion' :['gini', 'entropy']
}

mod = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 10,n_jobs=-1)
######################################################################


predict_model = mod.fit(d2_train_dataset,y_train)
print('model best parameter : ',predict_model.best_params_)
print('fitting ',mode,' is complete...')
print(mode,'score is :',predict_model.score(d2_test_dataset,y_test))

prediction = predict_model.predict(d2_test_dataset)



hit = 0
miss = 0
for x in range(len(y_test)):
    if prediction[x] == y_test[x]:
        hit+=1
    else:
        miss+=1

print('Validation Result...')
print('hit: ',hit,' miss : ',miss)

print('\n\nnow test new data..')

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp.append(df['GX'][i]/1000)
        tmp.append(df['GY'][i]/1000)
        tmp.append(df['GZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
total_data = np.array(total_data)
nsamples, nx, ny = total_data.shape
sample = total_data.reshape((nsamples,nx*ny))

sample_pred = predict_model.predict(sample)

lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(total_label[x],' --> ' ,lab[x],'err_index : ',x)

print('hit: ',hit,' miss : ',miss)

In [ ]:
#위에거는 랜덤 포레스트

In [ ]:
print('parameter Searching....')
#####here to change####
######################################################################

leaf_size = list(range(1,50))
n_neighbors = list(range(1,8))
p=[1,2]

hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

knn = KNeighborsClassifier()

mod = GridSearchCV(knn, hyperparameters, cv=10, n_jobs=-1)

######################################################################


predict_model = mod.fit(d2_train_dataset,y_train)
print('fitting ',mode,' is complete...')
print(mode,'score is :',predict_model.score(d2_test_dataset,y_test))


prediction = predict_model.predict(d2_test_dataset)

print(classification_report(prediction ,y_test))

hit = 0
miss = 0
for x in range(len(y_test)):
    if prediction[x] == y_test[x]:
        hit+=1
    else:
        miss+=1

print('Validation Result...')
print('hit: ',hit,' miss : ',miss)

print('\n\nnow test new data..')

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp.append(df['GX'][i]/1000)
        tmp.append(df['GY'][i]/1000)
        tmp.append(df['GZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
total_data = np.array(total_data)
nsamples, nx, ny = total_data.shape
sample = total_data.reshape((nsamples,nx*ny))

sample_pred = predict_model.predict(sample)

lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(total_label[x],' --> ' ,lab[x],'err_index : ',x)

print('hit: ',hit,' miss : ',miss)

In [ ]:
## KNN